In [52]:
import html5lib.treebuilders.base
html5lib.treebuilders._base= html5lib.treebuilders.base
from bs4 import BeautifulSoup
import pandas

In [53]:
text_file = open('resources/scraped/1/1.html', 'r')
html = text_file.read()


In [54]:
##Testing code for dataframe construction (empty and single row and their concatenation)
mcollector = pandas.DataFrame(columns=['messageid','message'])
mtreecollector = pandas.DataFrame(columns=['pmsgid','cmsgid'])
msgid=1 #initialize message id as 1.
# newone = pandas.DataFrame({'messageid' : [1.,2.], 'message' : ['test message','dd']})
# f = collector.append(newone)
# print(f)

In [55]:
def addMessage(msgid,msg,mcollector):
    #add msgid and msg to mcollector and return the modifed dataframe 
    #create a single row dataframe
    newone = pandas.DataFrame({'messageid':[msgid], 'message':[msg]})
    #append to mcollector and return the modified one
    return mcollector.append(newone)

In [56]:
def addParentChildRelation(pmsgid,cmsgid,mtreecollector):
    #add parent's msgid and child's msgi to mtreecollector and return the modified dataframe
    #create a single row dataframe
    newone = pandas.DataFrame({'pmsgid':[pmsgid], 'cmsgid':cmsgid})
    #append to mtreecollector and return the modified one
    return mtreecollector.append(newone)

In [57]:
s= BeautifulSoup(html,"lxml")

# If parent.parent.attrs is xg_module_body then it means it is the topmost question
# If parent.parent.attrs is i0,i1 etc. then we need to establish the ordering of 
# questions and answers. FOr more info run the following commented code.
# for descriptions in s.select('div.description'):
#     print(descriptions.parent.parent.attrs['class'])

lastOccurrences={}
global msgid, mcollector,mtreecollector
# Find all <p> tags in div.description and extract their text. The concatenated result
# will form the body of the reply (question or answer). For more info run the following code.
for description in s.select('div.description'):
    #Get the content of the description in a variable 'res'
    res=""
    for pars in description.find_all('p'):
        res = res+'\n'+(pars.getText())
    #find the parent.parent.attrs of this description and set it as question/answer
    #accordingly
    if description.parent.parent.attrs['class'][0] == "xg_module_body":
        #means it is topmost question
        res='QUESTION: \n'+res
        #get anew message id
        msgid=msgid+1
        #add message id and res in the message table.
        mcollector = addMessage(msg=res,msgid=msgid,mcollector=mcollector)
        #set this message id for -1 in lastOccurrence dictionary.
        lastOccurrences[-1]=msgid
    elif description.parent.parent.attrs['class'][0] == "discussion":
        #means it is one of the discussion, now extract it's id and based on that 
        #set it as a reply to previous question of a new question.
        res='DISCUSSION:\n'+res
        #extract its number
        num = description.parent.parent.attrs['class'][2]
        num=int(num[len(num)-1])
        #get a new message id
        msgid=msgid+1
        #add message id and res in the message table.
        mcollector = addMessage(msg=res, msgid=msgid,mcollector=mcollector)
        #get it's parent's infonumber (to handle 0, set the original question's message id
        #at -1)
        parentinfonumber = num-1
        assert parentinfonumber in lastOccurrences, 'Some serious error as the parent entry is not yet added'
        parentmsgid = lastOccurrences[parentinfonumber]        
        #add parentmessageid and this message id in messagetree table.
        mtreecollector = addParentChildRelation(pmsgid=parentmsgid,cmsgid=msgid,mtreecollector=mtreecollector)
        #set this message id for num in lastOccurence dictionary.
        lastOccurrences[num]=msgid
    else:
            print('Serious error: Not accounted for')
    #print(res+'\n-------------------------------------------\n')

print(msgid)
#write dataframes to csv files.
#print(mcollector)
mcollector.to_csv('resources/scraped/messages.csv',index=False)
mtreecollector.to_csv('resources/scraped/mtree.csv',index=False)



7
